In [1]:
"""
langchain Tools.
Tools are interfaces that an agent,chain,or LLM can use to interact with the world.They combine few things.
1. name of the tool.
2. A description of what the tool is.
3. JSON schema of what the inputs to the tool are.
4. The function to call.
5. whether the result of a tool should be returned directly to the user.
"""

'\nlangchain Tools.\nTools are interfaces that an agent,chain,or LLM can use to interact with the world.They combine few things.\n1. name of the tool.\n2. A description of what the tool is.\n3. JSON schema of what the inputs to the tool are.\n4. The function to call.\n5. whether the result of a tool should be returned directly to the user.\n'

In [29]:
import os
from dotenv import load_dotenv

# wikipedia tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# to read the content from website we will use this.
from langchain_community.document_loaders import WebBaseLoader


from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# creating retrieval tool
from langchain.tools.retriever import create_retriever_tool

# Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

# agents 
from langchain.agents import create_openai_tools_agent

# for llm
from langchain_openai import ChatOpenAI

# for prompt creation
from langchain import hub

# Agent executer
from langchain.agents import AgentExecutor

In [24]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [14]:
# This will interact with the wikipedia.
# Tool 1 wikipedia tool.
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [15]:
wiki.name

'wikipedia'

In [9]:
web_base_loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = web_base_loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029DC3EF3D60>)

In [11]:
# Tool 2 langsmith search.
retrieval_tool = create_retriever_tool(retriever, "langsmith_search", 
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
retrieval_tool.name

'langsmith_search'

In [13]:
# Tool 3 Arxiv
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper) 
arxiv.name

'arxiv'

In [18]:
# combine all the tools
tools = [wiki, arxiv, retrieval_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\AnuragReddy_Naredla_python_data\\python\\generative_ai\\langchain\\langchain_projects\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029DC5746950>, retriever=VectorStoreRetriever(ta

In [25]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [27]:
# langchain crteated own prompts we are using that one.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
# quering from all the above tools.
"""
The core idea of agents is to use a language model to choose a sequence of actions to take.In chains, a sequence of actions is 
hardcoded(in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
"""
agent = create_openai_tools_agent(llm, tools, prompt)

In [30]:
# agent executer
# if verbose = True we will get all the details.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [31]:
# This que will interact with the tool langsmith
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith​We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API key​To create an API key head to the setting pages. Then click Create API Key.Setup your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the 

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring you can ship quickly and with confidence. LangSmith offers Python and Typescript SDKs for all your LangSmith needs. You can create an API key, set up your environment, and log your first trace using LangSmith. \n\nLangSmith supports various workflows at each stage of the LLM application lifecycle and provides features like tracing, evaluation, pricing information, self-hosting options, and a Prompt Hub for prompt management. Additionally, LangSmith offers a Cookbook with tutorials and walkthroughs, along with Discord for community discussions.\n\nIf you have sensitive data that cannot be logged or if you need a private deployment of LangSmith, you can reach out to LangChain for self-hosting options that require an annual enterprise license.\n\nFor more detailed information

In [32]:
agent_executor.invoke({"input":"Tell me about ML"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Machine Learning`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating and using algorithms that allow computers to learn from and make predictions or decisions based on data. ML is widely used in various applications such as image and speech recognition, medical diagnosis, recommendation systems, and more.

> Finished chain.


{'input': 'Tell me about ML',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating and using algorithms that allow computers to learn from and make predictions or decisions based on data. ML is widely used in various applications such as image and speech recognition, medical diagnosis, recommendation systems, and more.'}

In [33]:
agent_executor.invoke({"input":"What is the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': 'What is the paper 1605.08386 about?',
 'output': 'The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}